In [1]:
import pandas as pd
import graphene
import extraction
import requests
import os
import json
import requests
import base58
import binascii
from tqdm import tqdm
import requests
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport

In [2]:
# Trad extract using requests for title, url, image
def extract(url):
    html = requests.get(url).text
    extracted = extraction.Extractor().extract(html, source_url=url)
    return extracted

In [3]:
extract('https://www.moesif.com/blog/technical/graphql/Getting-Started-with-Python-GraphQL-Part1/')

<Extracted: (title: 'Getting Started with Python and GraphQL - Part 1', 5 more), (url: 'https://www.moesif.com/blog/technical/graphql/Gett'), (image: 'https://blog.moesif.com/images/posts/technical/gra', 8 more), (description: 'Getting started with GraphQL & Python using Django', 5 more)>

In [9]:
USER_RESERVES_QUERY = """
{{
  userReserves(where: {{ user: "{address}"}}) {{
    id
    reserve{{
      id
      symbol
    }}
    user {{
      id
    }}
  }}
}}"""
url = 'https://api.thegraph.com/subgraphs/name/aave/protocol-raw'
transport = RequestsHTTPTransport(url=url, retries=5)

try:
    client = Client(transport=transport, fetch_schema_from_transport=True)
except (requests.exceptions.RequestException) as e:
    raise ValueError(f'Failed to connect to the graph at {url} due to {str(e)}') from e

address = '0xd791427e372186E90637caF1D6D6AFb054BeeC7A'
query = client.execute(gql(USER_RESERVES_QUERY.format(address=address.lower())),)

In [2]:
def TestGql(url):
    transport = RequestsHTTPTransport(url=url, retries=5)

    try:
        client = Client(transport=transport, fetch_schema_from_transport=True)
    except (requests.exceptions.RequestException) as e:
        raise ValueError(f'Failed to connect to the graph at {url} due to {str(e)}') from e

    query = gql(
            """
            {
            punks(first: 5) {
                id
                owner {
                id
                }
                totalOwners
                highestPrice
                sales {
                id
                }
            }
            accounts(first: 5) {
                id
                totalSpentBuyingPunks
                totalEarnedSellingPunks
                totalPunks
            }
            }
            """
        )

    # Execute the query on the transport
    result = client.execute(query)
    return result

In [3]:
TestGql(url='https://api.thegraph.com/subgraphs/name/crazyrabbitltc/punk-tools')

{'accounts': [{'id': '0x0000000000000000000000000000000000000000',
   'totalEarnedSellingPunks': '0',
   'totalPunks': '3981',
   'totalSpentBuyingPunks': '0'},
  {'id': '0x0000000000000000000000000000000000000001',
   'totalEarnedSellingPunks': '0',
   'totalPunks': '1',
   'totalSpentBuyingPunks': '0'},
  {'id': '0x0000000000000000000000000000000000000002',
   'totalEarnedSellingPunks': '0',
   'totalPunks': '0',
   'totalSpentBuyingPunks': '0'},
  {'id': '0x0000000000000000000000000000000000000003',
   'totalEarnedSellingPunks': '0',
   'totalPunks': '0',
   'totalSpentBuyingPunks': '0'},
  {'id': '0x0000000000000000000000000000000000000004',
   'totalEarnedSellingPunks': '0',
   'totalPunks': '0',
   'totalSpentBuyingPunks': '0'}],
 'punks': [{'highestPrice': '0',
   'id': '0',
   'owner': {'id': '0x0000000000000000000000000000000000000000'},
   'sales': [{'id': '0x24893e1197d9da51601987636e2c2bea6d7e56391eb410b44ff5aa8ea80fc068-punkBought'},
    {'id': '0x54c8c5d3762eb02d657034873

In [92]:
def TestGql(url):
    transport = RequestsHTTPTransport(url=url, retries=5)

    try:
        client = Client(transport=transport, fetch_schema_from_transport=True)
    except (requests.exceptions.RequestException) as e:
        raise ValueError(f'Failed to connect to the graph at {url} due to {str(e)}') from e

    query = gql(
            """
            {
            contracts {
                id
                uri
                name
                symbol
            }
            nfts(orderBy: avgSaleWei, orderDirection: desc) {
                tokenId
                uri
                metadata
                creator {
                totalCreations
                totalCreationsSold
                }
                sales(orderBy: timestamp, orderDirection: desc) {
                amount
                timestamp
                buyer {
                    id
                    totalBoughtWei
                }
                seller {
                    id
                    totalSoldWei
                }
                }
            }
            }
            """
        )

    # Execute the query on the transport
    result = client.execute(query)
    return result

In [93]:
TestGql(url='https://api.thegraph.com/subgraphs/name/upshot-tech/nft-analytics-superrare')

{'contracts': [{'id': '0x41a322b28d0ff354040e2cbc676f0320d8c8850d',
   'name': 'SuperRare',
   'symbol': 'SUPR',
   'uri': 'https://www.superrare.co/'}],
 'nfts': [{'creator': {'totalCreations': '89', 'totalCreationsSold': '41'},
   'metadata': '{"name":"Anxiety","description":"XCOPY, 2014","tags":["XCOPY"," GIF"," anxiety"," panic"," pain"],"image":"https://ipfs.pixura.io/ipfs/QmXAYvTCi4a1k3SDp18Xg6RcwUyAtLqQ7roz9sdKeFfqM6"}',
   'sales': [{'amount': '1000000000000000000000',
     'buyer': {'id': '0xf476cd75be8fdd197ae0b466a2ec2ae44da41897',
      'totalBoughtWei': '1626990000000000000000'},
     'seller': {'id': '0x215fedd5780c39fbdadb4bc3700de61a8ffdd74a',
      'totalSoldWei': '1000000000000000000000'},
     'timestamp': '1616589182'}],
   'tokenId': '14',
   'uri': 'https://ipfs.pixura.io/ipfs/QmQq6EU4Bh2Cyv5YeCwHBUWH59TFCtvBsroMPQfzqkPGJ4'},
  {'creator': {'totalCreations': '89', 'totalCreationsSold': '41'},
   'metadata': '{"name":"All Time High in the City","description":"XCOPY

In [89]:
punk_list = []

for i in tqdm(range(0,100,1)): #pages 2 through 450
    punks = TestGql(url='https://api.thegraph.com/subgraphs/name/itsjerryokolo/cryptopunks', punk_id=(i))
    punk_list.append(punks)


100%|██████████| 100/100 [00:27<00:00,  3.61it/s]


In [90]:
punk_list

[{'purchases': [{'amount': '0',
    'buyer': {'id': '0x0000000000000000000000000000000000000000'},
    'id': '0',
    'punk': {'id': '0'},
    'seller': '0xf5099e196469257c0d457e896a648d5f616af034',
    'transaction': {'id': '0x5687d030ddf9059d8098f6095e679085dd153ceb6d5b2f45ac36c16876de94e5'}}]},
 {'purchases': [{'amount': '0',
    'buyer': {'id': '0x0000000000000000000000000000000000000000'},
    'id': '1',
    'punk': {'id': '1'},
    'seller': '0xee3766e4f996dc0e0f8c929954eaafef3441de87',
    'transaction': {'id': '0xf4af5563f3c4c3b26dee3ab027902f113bee5985b28d9ede5b81ab42b46abb30'}}]},
 {'purchases': []},
 {'purchases': []},
 {'purchases': []},
 {'purchases': []},
 {'purchases': []},
 {'purchases': []},
 {'purchases': []},
 {'purchases': []},
 {'purchases': []},
 {'purchases': []},
 {'purchases': []},
 {'purchases': []},
 {'purchases': [{'amount': '2500000000000000000',
    'buyer': {'id': '0x7c00c9f0e7aed440c0c730a9bd9ee4f49de20d5c'},
    'id': '14',
    'punk': {'id': '14'},
   

In [86]:
pd.DataFrame(punk_list)

,purchases
0,"[{'amount': '0', 'buyer': {'id': '0x0000000000..."
1,"[{'amount': '0', 'buyer': {'id': '0x0000000000..."
2,[]
3,[]
4,[]
5,[]
6,[]
7,[]
8,[]
9,[]


In [71]:
pd.concat({k: pd.DataFrame(v) for k, v in foo.items()}, axis=0)

,,amount,buyer,id,punk,seller,transaction
purchases,0,50020000000000000000,{'id': '0x701f373df763308d96d8537822e8f9b2bae4...,3392,{'id': '3392'},0x82898337f03c1d73133213580b0d9855d3a5573f,{'id': '0xf2fc78a37c0a50febd02181cdf386d531c0b...
